## Auditing and Exporting Detections

#### Before running this script, make sure that your Google Drive folder contains the orthomosaic GeoTiff (`step 0`) and no other GeoTIFF files, and the `data.json` file you created (`step 1`) and the `new_detections.json` file you created (`step 4`). If you want to add your new CNN detections to the manually annotated detections you created in VIA, also add the JSON file you exported using VIA in `step 2`. You will need to input that file name directly, since it is not standardized in our workflow.

<a href="https://colab.research.google.com/github/gl7176/GreySealCNN/blob/master/5_export_detections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
#####  <center> Be sure to update this hyperlink above if you clone and want to point to a different GitHub </center>

In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1INuRNVKvKMy8L_Nb6lmoVbyvScWK0-0D'

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls every file in the directory specified above
orthomosaic_file = {}
count = 0
for f in file_list:
  count += 1
  if count % 10 == 0:
    print(count)
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  if fname.endswith(".tif") or fname.endswith(".json"):
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
    os.stat(fname)
    # if the file is a *.tif and larger than 100 mb we label it the orthomosaic
    if fname.endswith(".tif") and os.stat(fname).st_size > 10^8 :
      # if there are multiple orthomosaic files detected we spit an error
      if len(orthomosaic_file) != 0:
        raise Exception("more than one orthomosaic file identified based on size and type")
      orthomosaic_file = fname
      print("orthomosaic identified as " + orthomosaic_file)
  

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230


In [ ]:
import os
import argparse
import numpy as np
import json
import csv
!pip install rasterio
import rasterio

!shapely.geometry import mapping, Polygon
!pip install fiona
import fiona # only required for exporting to shapefiles

/bin/bash: shapely.geometry: command not found
     |████████████████████████████████| 14.8MB 322kB/s 


In [ ]:
# ingest the image
infile = orthomosaic_file

img_dir = infile.split(".")[0]
prj_name = img_dir.split("/")[-1]
data_dir = img_dir.split("/")[0]

In [ ]:
# open the output from seal_detection

with open(local_download_path + '/new_detections.json') as f:
    detected_labels = json.load(f)

### Exporting to Shapefile

In [98]:
image_annotations = []
for key, value in detected_labels.items():
    #print(key)
    #print(value)
    annotation = [[key][0].split("/")[-1]]
    print(annotation)
    detections = []
    for item in value:
        box_w_label = []
        box = item['box']
        box_w_label.append(box)
        score = item['score']
        box_w_label.append(score)
        label = item['label']
        box_w_label.append(label)
        detections.append(box_w_label)
        #print(item)
    annotation.append(detections)
    image_annotations.append(annotation)
print(image_annotations)

['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---93.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---141.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---47.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---46.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---191.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---225.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---140.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---131.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---212.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---196.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---170.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---92.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---178.png']
['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---155.pn

In [99]:
scores = []
for key, value in detected_labels.items():
    for item in value:
        score = item['score']
        scores.append(score)

In [108]:
with open(local_download_path + '/data.json') as f:
  img_data = json.load(f)

image_bbox = []
for annotation in image_annotations:
    for detection in annotation[1]:
        try:
            image_box_list = []
            local_bounding_box = np.array([[detection[0][0], detection[0][1]], [detection[0][2], detection[0][1]], [detection[0][2], detection[0][3]], [detection[0][0], detection[0][3]]]).astype(int)
            image_located_bb = local_bounding_box + [img_data["image_locations"][annotation[0]]]
            image_box_list.append(image_located_bb)
            image_box_list.append(detection[1])
            image_box_list.append(detection[2])
            image_bbox.append(image_box_list)
            
        except ValueError: # if the image doesn't have a detection
            pass

In [111]:
print(image_bbox)[1]

[[array([[8066, 4623],
       [8112, 4623],
       [8112, 4655],
       [8066, 4655]]), 0.8100398778915405, 1], [array([[8042, 4641],
       [8079, 4641],
       [8079, 4677],
       [8042, 4677]]), 0.796779215335846, 1], [array([[8037, 5080],
       [8086, 5080],
       [8086, 5156],
       [8037, 5156]]), 0.7938922047615051, 0], [array([[8278, 4850],
       [8319, 4850],
       [8319, 4886],
       [8278, 4886]]), 0.7647950053215027, 1], [array([[8113, 4747],
       [8154, 4747],
       [8154, 4785],
       [8113, 4785]]), 0.7569835782051086, 1], [array([[7892, 4779],
       [7942, 4779],
       [7942, 4817],
       [7892, 4817]]), 0.7454524040222168, 1], [array([[7840, 4848],
       [7886, 4848],
       [7886, 4886],
       [7840, 4886]]), 0.7198755145072937, 1], [array([[7953, 5466],
       [8040, 5466],
       [8040, 5508],
       [7953, 5508]]), 0.7141700983047485, 0], [array([[8199, 5346],
       [8254, 5346],
       [8254, 5414],
       [8199, 5414]]), 0.7027513384819031, 0], [

TypeError: ignored

In [ ]:
bbox = []
for annotation in image_bbox:
    x1 = annotation[0][0]
    y1 = annotation[0][1]
    x2 = annotation[1][0]
    y2 = annotation[2][1]
    bounding_box = [x1,y1,x2,y2]
    bbox.append(bounding_box)

In [ ]:
# Malisiewicz et al.
# import the necessary packages
import numpy as np

def non_max_suppression(boxes, probs=None, overlapThresh=0.6):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    # if probabilities are provided, sort on them instead
    if probs is not None:
        idxs = probs

    # sort the indexes
    idxs = np.argsort(idxs)

    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return only the bounding boxes that were picked
    return boxes[pick].astype("int")

In [ ]:
bboxes = np.array(bbox)
pick = non_max_suppression(bboxes, scores, 0.6)
pick_list = pick.tolist()

In [ ]:
# ingest back in the coordinates of detections within an image referenced by their filename

with open(local_download_path + '/data.json') as f:
    img_data = json.load(f)

# open the satellite image
dataset = rasterio.open(infile)

geolocated_annotations_before_nms = []

for annotation in image_annotations:
    for detection in annotation[1]:
        try:            
            local_bounding_box = np.array([[detection[0], detection[1]], [detection[2], detection[1]], [detection[2], detection[3]], [detection[0], detection[3]]]).astype(int)
            image_located_bb = local_bounding_box + [img_data["image_locations"][annotation[0]]]
            
            geolocated_bb = []
            for point in image_located_bb:
                geolocated_bb.append(dataset.transform * point)
            geolocated_annotations_before_nms.append(geolocated_bb)
        except ValueError: # if the image doesn't have a detection
            pass

geolocated_annotations_after_nms = []
        
for box in pick_list:
    image_located_bb = np.array([[box[0], box[1]], [box[2], box[1]], [box[2], box[3]], [box[0], box[3]]]).astype(int)
            
    geolocated_bb = []
    for point in image_located_bb:
        geolocated_bb.append(dataset.transform * point)
    geolocated_annotations_after_nms.append(geolocated_bb)

In [44]:
print(pick_list)

[[8136, 11261, 8179, 11294], [8419, 10932, 8473, 11012], [7392, 9367, 7463, 9431], [7863, 9912, 7932, 9979], [3714, 12101, 3754, 12136], [9175, 7604, 9253, 7653], [11276, 2059, 11318, 2094], [12840, 2365, 12916, 2415], [6873, 5117, 6915, 5151], [11996, 2700, 12041, 2736], [8066, 4623, 8112, 4655], [4638, 7455, 4700, 7514], [4369, 10745, 4420, 10813], [7389, 11260, 7473, 11316], [8042, 4641, 8079, 4677], [8037, 5080, 8086, 5156], [8173, 11119, 8216, 11154], [8931, 7371, 9009, 7415], [12514, 2155, 12586, 2202], [7830, 9544, 7876, 9580], [8278, 4850, 8319, 4886], [12723, 2084, 12774, 2155], [4842, 8192, 4902, 8250], [8113, 4747, 8154, 4785], [4941, 8175, 5013, 8234], [11875, 2207, 11935, 2256], [11328, 1966, 11371, 2002], [3862, 10297, 3929, 10349], [11860, 1979, 11897, 2013], [4753, 8084, 4822, 8143], [7892, 4779, 7942, 4817], [7915, 9789, 7974, 9856], [8075, 11121, 8120, 11155], [7478, 9277, 7532, 9342], [7698, 9296, 7739, 9330], [3912, 12492, 3992, 12543], [7647, 9238, 7688, 9273], [12

In [23]:
print("before NMS: " + str(len(geolocated_annotations_before_nms)))
print("after NMS: " + str(len(geolocated_annotations_after_nms)))

before NMS: 132
after NMS: 126


In [ ]:
len(geolocated_annotations_after_nms)

98

In [ ]:
# write out the detections as a shapefile

from collections import OrderedDict
import fiona
from fiona.crs import from_epsg

# Set output directory
output_dir = 'shapefile_output'

# create the dir if it doesn't already exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str')
  ])
}

with fiona.open(output_dir + '/seal_detections.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(geolocated_annotations_after_nms):
      print (num, polygon)
        record = {
            'geometry': {'coordinates': [polygon], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', infile),
                                       ('Detection', 'Albatross')
                                       ]),
            'type': 'Feature'}
        c.write(record)

In [42]:
for num, polygon in enumerate(geolocated_annotations_after_nms):
      print (num, polygon)

0 [(292130.18141, 5099948.34892), (292131.72640000004, 5099948.34892), (292131.72640000004, 5099947.16323), (292130.18141, 5099947.16323)]
1 [(292140.3496, 5099960.1698900005), (292142.28982, 5099960.1698900005), (292142.28982, 5099957.29549), (292140.3496, 5099957.29549)]
2 [(292103.44949, 5100016.40034), (292106.00052, 5100016.40034), (292106.00052, 5100014.10082), (292103.44949, 5100014.10082)]
3 [(292120.37252000003, 5099996.81849), (292122.85169000004, 5099996.81849), (292122.85169000004, 5099994.41118), (292120.37252000003, 5099994.41118)]
4 [(291971.29895, 5099918.16772), (291972.73615, 5099918.16772), (291972.73615, 5099916.91017), (291971.29895, 5099916.91017)]
5 [(292167.51268000004, 5100079.74493), (292170.31522000005, 5100079.74493), (292170.31522000005, 5100077.98436), (292167.51268000004, 5100077.98436)]
6 [(292243.00161000004, 5100278.97678), (292244.51067000005, 5100278.97678), (292244.51067000005, 5100277.71923), (292243.00161000004, 5100277.71923)]
7 [(292299.19613000

### Zip output folder for download

In [20]:
# zip up the output directory into an archive for download
import subprocess
subprocess.call(['zip', '-r', '/content/' + output_dir + '.zip', '/content/' + output_dir])

from google.colab import files
files.download("/content/" + output_dir + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### RetinaNet to Existing VIA

In [ ]:
# open the previous training data

with open(data_dir + '/via_SealCNN_TrainingData.json') as f:
    existing_labels = json.load(f)

# add the new detections to the old via_region_data.json file

#"11_fiX1mEhK","[""2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---27.png""]",0,"[]","[2,509.275,929.174,89.376,48.904]","{}"

for filepath, detections in detected_labels.items():
    fn = filepath.split("/")[-1]
    # TODO is deep copy correct?
    for filename_size, metadata in existing_labels.items():
        if fn == metadata["filename"]:
            for detection in detections:
                # 'box' : [x1, y1, x2, y2]
                x1 = detection["box"][0]
                y1 = detection["box"][1]
                x2 = detection["box"][2]
                y2 = detection["box"][3]
                #print(x1,x2,y1,y2)
                metadata["regions"].append({'shape_attributes': {'name': 'rect', 'x': x1, 'y': y1, 'width': x2-x1, 'height': y2-y1}, 'region_attributes': {}})
    

In [ ]:
# write out new VIA file with additional detections

with open(img_dir + '/via_region_data_detections.json', 'w') as fp:
    json.dump(existing_labels, fp)